# Introduction

[Start here](https://huggingface.co/learn/mcp-course/en/unit1/key-concepts)

# Key Concepts and Terminology

MCP is often described as the "USC=C for AI applications." Just as USB-C provides a standardized physical and logical interface for connecting various peripherals to computing devices, MCP offers a consistent protocol for linking AI models to external capabilities. This standardization benefits the entire ecosystem:
* users enjoy simpler and more consistent experiences across AI applications
* AI application developers gain easy integration with a growing ecosystem of tools and data sources
* tool and data providers need only create a single implementation that works with multiple AI applications
* the broader ecosystem benefits from increased interoperability, innovation, and reduced fragmentation

## The Integration Problem

The **MxN Integration Problem** refers to the challenge of connecting M different AI applications to N different external tools or data sources without a standardized approach.

## Without MCP (M×N Problem)

Without a protocol like MCP, developers would need to create M×N custom integrations—one for each possible pairing of an AI application with an external capability.

Each AI application would need to integrate with each tool/data source individually. This is a very complex and expensive process which introduces a lot of friction for developers, and high maintenance costs.

Once we have multiple models and multiple tools, the number of integrations becomes too large to manage, each with its own unique interface.

## With MCP (M+N Solution)

MCP transforms this into an M+N problem by providing a standard interface: each AI application implements the client side of MCP once, and each tool/data source implements the server side once. This dramatically reduces integration complexity and maintenance burden.

## Core MCP Terminology

MCP is a standard like HTTP or USB-C, and is a protocol for connecting AI applications to external tools and data sources. Therefore, using standard terminology is crucial to making the MCP work effectively.

When documenting our applications and communicating with the community, we should use the following terminology.

Just like client server relationships in HTTP, MCP has a client and a server.

* **Host:** The user-facing AI application that end-users interact with directly. Examples include Anthropic’s Claude Desktop, AI-enhanced IDEs like Cursor, inference libraries like Hugging Face Python SDK, or custom applications built in libraries like LangChain or smolagents. Hosts initiate connections to MCP Servers and orchestrate the overall flow between user requests, LLM processing, and external tools.

* **Client:** A component within the host application that manages communication with a specific MCP Server. Each Client maintains a 1:1 connection with a single Server, handling the protocol-level details of MCP communication and acting as an intermediary between the Host’s logic and the external Server.

* **Server:** An external program or service that exposes capabilities (Tools, Resources, Prompts) via the MCP protocol.

[Terminology](https://huggingface.co/learn/mcp-course/en/unit1/key-concepts#capabilities)

**Then there's a bunch of other stuff you're better off just reading instead of typing along to.**

# The Communication Protocol

MCP defines a standardized communication protocol that enables Clients and Servers to exchange messages in a consistent, predictable way. This standardization is critical for interoperability across the community.

## JSON-RPC: The Foundation